In [ ]:
import os
import requests
import datetime
import tweepy

def fetch_fpl_data():
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    return data['elements'], data['teams']

def get_team_name_map(teams):
    return {team['id']: team['short_name'] for team in teams}

def format_delta(n):
    symbol = "△" if n > 0 else "▽"
    abs_n = abs(n)
    return f"{symbol} {abs_n/1_000_000:.1f}m" if abs_n >= 1_000_000 else f"{symbol} {abs_n/1000:.1f}K"

def generate_risers_and_fallers(players, team_map):
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    enriched = []
    for p in players:
        ownership_pct = float(p['selected_by_percent'])
        if ownership_pct >= 2.0:
            enriched.append({
                'name': f"{p['first_name']} {p['second_name']}",
                'price': p['now_cost'] / 10,
                'team': team_map[p['team']],
                'ownership_pct': ownership_pct,
                'ownership_change': p['transfers_in_event'] - p['transfers_out_event']
            })

    sorted_players = sorted(enriched, key=lambda x: x['ownership_change'], reverse=True)
    top10 = sorted_players[:3]
    bottom10 = sorted_players[-3:]

    risers = [f"FPL Risers | {today}\n"]
    for p in top10:
        delta = format_delta(p['ownership_change'])
        risers.append(f"{p['name']} ({p['team']}) - £{p['price']} - {p['ownership_pct']:.1f}% - {delta}")

    fallers = [f"\nFPL Fallers | {today}\n"]
    for p in bottom10:
        delta = format_delta(p['ownership_change'])
        fallers.append(f"{p['name']} ({p['team']}) - £{p['price']} - {p['ownership_pct']:.1f}% - {delta}")

    return "\n".join(risers + fallers)

def post_to_x(text):
    # Replace with your actual credentials

    consumer_key=""
    consumer_secret=""
    access_token=""
    access_token_secret=""

    auth = tweepy.OAuth1UserHandler(
        consumer_key, consumer_secret,
        access_token, access_token_secret
    )
    api = tweepy.API(auth)

    if len(text) <= 280:
        api.update_status(text)
    else:
        chunks = []
        lines = text.split("\n")
        chunk = ""
        for line in lines:
            if len(chunk + "\n" + line) > 280:
                chunks.append(chunk.strip())
                chunk = line
            else:
                chunk += "\n" + line
        chunks.append(chunk.strip())

        tweet = api.update_status(chunks[0])
        for chunk in chunks[1:]:
            tweet = api.update_status(chunk, in_reply_to_status_id=tweet.id, auto_populate_reply_metadata=True)

    print("✅ Risers & Fallers posted to X!")

if __name__ == "__main__":
    try:
        players, teams = fetch_fpl_data()
        team_map = get_team_name_map(teams)
        report = generate_risers_and_fallers(players, team_map)
        post_to_x(report)
    except Exception as e:
        print(f"❌ Error: {e}")
        exit(1)


❌ Error: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product
